In [1]:
import os
import json
import re
import pandas as pd
from collections import OrderedDict

In [2]:
def get_end_salvo(versao, path = None):
    
    if path is None:
        path = '../Dados_gerados/geocode'
    if os.path.exists(path):
        pat = 'enderecos_georref_\d*.json'
        files = [os.path.join(path, file) for file in os.listdir(path)
                if re.match(pat, file)]

        for f in files:
            versao_file = os.path.split(f)[-1].split('_')[-1].split('.')[0]
            if str(versao) == versao_file:
                with open(f) as file:
                    return json.load(file)
    
    
    return None


def dados_banco_dados():
    
    path_dados = '../Dados_originais/Dados_Banco_de_Dados'
    files_site = [os.path.join(path_dados, file) for file in os.listdir(path_dados)]
    
    contrib_f = [f for f in files_site if f.split('\\')[-1].startswith('contrib')][0]
    prop_f = [f for f in files_site if f.split('\\')[-1].startswith('propost')][0]
    
    return pd.read_csv(contrib_f), pd.read_csv(prop_f)

In [3]:
def report_cidades(enderecos):
    
    results = enderecos.values()
    print(f'Total de enderecos : {len(results)}')
    cidades = {}
    for result in results:
        #pode ter mais de um endereço
        for end in result:
            for component in end['address_components']:
                if 'administrative_area_level_2' in component['types']:
                    nom_cidade = component['long_name']
                    if nom_cidade not in cidades:
                        cidades[nom_cidade] = 0
                    cidades[nom_cidade] +=1
    for key, value in cidades.items():
        print(f'{key} : {value}')

In [4]:
def get_component_by_type(comp, type_name):
    
    if type_name in comp['types']:
        return comp['long_name']
    else:
        return None

def parse_address_comps(address):
    
    de_para = OrderedDict(
        {
            'bairro' : 'sublocality_level_1',
            'cidade' : 'administrative_area_level_2',
            'estado' : 'administrative_area_level_1',
            'pais' : 'country',
            
        }
    )
    
    dados = {}
    for nom_meu, nom_google in de_para.items():
        #nao consegui pensar em uma otimizacao, infelizmente,
        # mas a ordem tende a seguir a do de-para
        for component in address['address_components']:
            parsed = get_component_by_type(component, nom_google)
            if parsed:
                dados[nom_meu] = parsed
                break
        else:
            dados[nom_meu] = None
    
    return dados

def parse_address(address):
    
    dados = {}
    dados.update(parse_address_comps(address))
    dados['endereco_completo'] = address['formatted_address']
    dados['tipo_endereco'] = '; '.join(address['types'])
    dados['latitude'] = address['geometry']['location']['lat']
    dados['longitude'] = address['geometry']['location']['lng']
    dados['tipo_localizacao'] = address['geometry']['location_type']
    
    return dados

def registro_vazio():
    
    return {
                'bairro' : None,
                'cidade' : None,
                'estado' : None,
                'pais' : None,
                'endereco_completo' : None,
                'tipo_endereco' : None,
                'latitude' : None,
                'longitude' : None,
                'tipo_localizacao' : None,
            }

def parse_all_geocode_data(enderecos):
    
    parsed_data = {}
    for end_original, results in enderecos.items():
        if len(results) == 1:
            parsed_data[end_original] = parse_address(results[0])
        elif len(results) > 1:
            print('Endereco com mais de um resultado:')
            print(end_original)
            #só tem um caso, achei ok pegar o primeiro resultado
            parsed_data[end_original] = parse_address(results[0])
        elif len(results) == 0:
            parsed_data[end_original] = registro_vazio()
        
    return parsed_data

def data_frame_geocod(parsed_data):
    
    df = pd.DataFrame(parsed_data).T
    df = df.reset_index().rename(
        {'index' : 'endereço_original'}, axis = 1)
    
    return df


def join_data(df, df_end, col_end):
    
    return pd.merge(
        df, df_end, 
         how = 'left',
         left_on = col_end, 
        right_on = 'endereço_original')

In [5]:
enderecos = get_end_salvo(versao = 1)
report_cidades(enderecos)


Total de enderecos : 254
São Paulo : 245
Guilford County : 1
São Vicente : 1
Bonif : 1
Osasco : 2
Guarulhos : 1
Jundiaí : 1


In [6]:
parsed = parse_all_geocode_data(enderecos)
df_parsed = data_frame_geocod(parsed)
df_parsed.to_excel('../Dados_gerados/enderecos_geocod.xlsx')

Endereco com mais de um resultado:
Rua Henry Fuseli 70 Viela são pedro São Paulo SP


In [7]:
df_parsed.sample(3)

,endereço_original,bairro,cidade,estado,pais,endereco_completo,tipo_endereco,latitude,longitude,tipo_localizacao
135,Rua Marechal Hermes da Fonseca 450 São Paulo SP,Santana,São Paulo,São Paulo,Brazil,"R. Mal. Hermes da Fonseca, 450 - Santana, São ...",street_address,-23.496195,-46.632332,ROOFTOP
83,Rua Osório Franco Vilhena 1099 São Paulo SP,Vila Nova Curuca,São Paulo,São Paulo,Brazil,"R. Osório Franco Vilhena, 1099 - Vila Nova Cur...",street_address,-23.507228,-46.418202,ROOFTOP
67,Rua Italina 295 CASA 03 São Paulo SP,Itaquera,São Paulo,São Paulo,Brazil,"R. Italina, 295 - CASA 03 - Itaquera, São Paul...",subpremise,-23.540795,-46.450408,ROOFTOP


In [8]:
contrib, prop = dados_banco_dados()
contrib_georref = join_data(contrib, df_parsed, col_end = 'Endereço')
prop_georref = join_data(prop, df_parsed, col_end = 'Endereco')


contrib_georref.to_excel('../Dados_gerados/contrib_georref.xlsx')
prop_georref.to_excel('../Dados_gerados/prop_georref.xlsx')